In [ ]:
import pandas as pd
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, Concatenate, Dropout, Conv1D, SpatialDropout1D, Add, Activation, GlobalAveragePooling1D, Reshape, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import os

# Configuration pour la reproductibilité
tf.random.set_seed(42)
np.random.seed(42)

# ==============================================================================
# 1. CHARGEMENT ET PRÉPARATION DES DONNÉES
# ==============================================================================
print("Chargement des données...")
# Assure-toi que le fichier est bien généré par ton script précédent
data = pd.read_csv('data/processed_data.csv')

# Conversion de la date
data['date'] = pd.to_datetime(data['date'])

# --- LOG-TRANSFORMATION DE LA CIBLE ---
# Critique pour la métrique RMSLE
data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

# --- NETTOYAGE ---
if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Définition des types de features
# On exclut 'id', 'sales', 'is_train', 'date'
ALL_COLS = [c for c in data.columns if c not in ['id', 'sales', 'is_train', 'date']]

# Features pour les Embeddings (Catégories à haute cardinalité ou cycliques)
CAT_COLS = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 'month', 'dayofweek']

# Features Numériques (Tout le reste : Lags, Rolling means, Oil, etc.)
NUM_COLS = [c for c in ALL_COLS if c not in CAT_COLS]

print(f"Nombre de features numériques pour le TCN : {len(NUM_COLS)}")
print(f"Nombre de features catégorielles : {len(CAT_COLS)}")

# ==============================================================================
# 2. PREPROCESSING (ENCODING & SCALING)
# ==============================================================================
print("Preprocessing...")

# A. Label Encoding pour les Embeddings
label_encoders = {}
for col in CAT_COLS:
    le = LabelEncoder()
    data[col] = data[col].astype(str) # Sécurité
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# B. Scaling des numériques (Indispensable pour les réseaux de neurones)
# On utilise StandardScaler pour centrer autour de 0, mieux pour les TCN
scaler = StandardScaler()
data[NUM_COLS] = scaler.fit_transform(data[NUM_COLS])
# Remplacer les NaN potentiels créés par le scaling (division par zero rare)
data[NUM_COLS] = data[NUM_COLS].fillna(0)

# ==============================================================================
# 3. SÉPARATION TRAIN / VALIDATION / TEST
# ==============================================================================
print("Séparation des jeux de données...")

train_df = data[data['is_train'] == 1].copy()
test_df = data[data['is_train'] == 0].copy()

# Validation temporelle (Les 15 derniers jours du train)
last_date = train_df['date'].max()
val_start = last_date - pd.DateOffset(days=15)

train_mask = train_df['date'] < val_start
val_mask = train_df['date'] >= val_start

# Création des arrays numpy pour Keras
def get_keras_data(df, num_cols, cat_cols):
    # 1. Input Numérique : Shape (N, Features)
    X_num = df[num_cols].values.astype('float32')
    
    # 2. Inputs Catégoriels : Liste de arrays (N, 1)
    X_cat = [df[c].values.astype('int32') for c in cat_cols]
    
    return X_num, X_cat

X_train_num, X_train_cat = get_keras_data(train_df[train_mask], NUM_COLS, CAT_COLS)
y_train = train_df.loc[train_mask, 'sales'].values.astype('float32')

X_val_num, X_val_cat = get_keras_data(train_df[val_mask], NUM_COLS, CAT_COLS)
y_val = train_df.loc[val_mask, 'sales'].values.astype('float32')

X_test_num, X_test_cat = get_keras_data(test_df, NUM_COLS, CAT_COLS)
test_ids = test_df['id'].values

# Nettoyage mémoire
del data, train_df, test_df
gc.collect()

# ==============================================================================
# 4. ARCHITECTURE TCN (TEMPORAL CONVOLUTIONAL NETWORK)
# ==============================================================================
print("Construction du modèle TCN...")

def residual_block(x, filters, kernel_size, dilation_rate, dropout_rate=0.1):
    """Bloc résiduel TCN standard : Conv1D dilatée + Skip Connection"""
    # Sauvegarde de l'entrée pour la connexion résiduelle
    shortcut = x
    
    # 1ère Conv Dilatée
    x = Conv1D(filters=filters, 
               kernel_size=kernel_size, 
               dilation_rate=dilation_rate, 
               padding='causal',  # Important pour ne pas voir le futur (même si ici on est sur des features)
               activation='relu')(x)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(dropout_rate)(x)
    
    # 2ème Conv (Optionnelle, ici on simplifie avec une seule par bloc pour la vitesse)
    
    # Adaptation de la dimension du shortcut si nécessaire (si le nb de filtres change)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(shortcut)
        
    # Addition (Skip Connection)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_tcn_model(num_features, cat_cols_info, label_encoders):
    inputs = []
    embeddings = []
    
    # --- BRANCHE 1 : EMBEDDINGS (Catégories) ---
    for col in cat_cols_info:
        vocab_size = len(label_encoders[col].classes_) + 1
        embed_dim = min(50, (vocab_size + 1) // 2) # Dimension raisonnable
        
        inp = Input(shape=(1,), name=f'input_{col}')
        inputs.append(inp)
        
        emb = Embedding(vocab_size, embed_dim)(inp)
        emb = Flatten()(emb)
        embeddings.append(emb)
    
    # Concaténation des embeddings
    x_cat = Concatenate()(embeddings)
    x_cat = Dense(64, activation='relu')(x_cat) # Pré-traitement des catégories
    
    # --- BRANCHE 2 : TCN (Numériques) ---
    # Input numérique standard (Batch, Num_Features)
    input_num = Input(shape=(num_features,), name='input_numeric')
    inputs.append(input_num)
    
    # TRUC : On reshape pour faire croire au TCN que c'est une séquence
    # On transforme (Batch, Features) -> (Batch, Features, 1)
    # Le TCN va "scanner" les features comme si c'était du temps
    x_tcn = Reshape((num_features, 1))(input_num)
    
    # Bloc TCN "Lite" (Petit pour tester vite)
    # Dilation rate augmente la fenêtre de vision : 1, 2, 4, 8...
    x_tcn = residual_block(x_tcn, filters=32, kernel_size=3, dilation_rate=1)
    x_tcn = residual_block(x_tcn, filters=32, kernel_size=3, dilation_rate=2)
    x_tcn = residual_block(x_tcn, filters=32, kernel_size=3, dilation_rate=4)
    
    # Pooling pour récupérer les infos les plus importantes des filtres
    x_tcn = GlobalAveragePooling1D()(x_tcn)
    
    # --- FUSION ---
    x = Concatenate()([x_cat, x_tcn])
    
    # Tête de prédiction (Dense)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    
    # Output (Régression linéaire)
    output = Dense(1, activation='linear', name='output')(x)
    
    model = Model(inputs=inputs, outputs=output)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

# Construction
model = build_tcn_model(len(NUM_COLS), CAT_COLS, label_encoders)
model.summary()

# ==============================================================================
# 5. ENTRAÎNEMENT
# ==============================================================================
print("\nDébut de l'entraînement...")

# Callbacks pour éviter le sur-apprentissage et optimiser le LR
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)
]

# Préparation des inputs sous forme de liste pour Keras [cat1, cat2, ..., num]
train_inputs = X_train_cat + [X_train_num]
val_inputs = X_val_cat + [X_val_num]

history = model.fit(
    train_inputs, y_train,
    validation_data=(val_inputs, y_val),
    epochs=15,          # Petit nombre pour tester vite (augmente à 30-50 si ça marche bien)
    batch_size=2048,    # Gros batch pour aller vite
    callbacks=callbacks,
    verbose=1
)

# ==============================================================================
# 6. PRÉDICTION ET SOUMISSION
# ==============================================================================
print("\nGénération des prédictions...")

test_inputs = X_test_cat + [X_test_num]
preds_log = model.predict(test_inputs, batch_size=2048).flatten()

# Inverse Log (expm1)
preds = np.expm1(preds_log)
preds[preds < 0] = 0 # Sécurité

submission = pd.DataFrame({'id': test_ids, 'sales': preds})
submission.to_csv('submission_tcn_lite.csv', index=False)

print("Terminé ! Fichier 'submission_tcn_lite.csv' généré.")